In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import PromptTemplate

from langchain.chains import RetrievalQA

In [2]:
## Read the ppdfs from the folder
loader=PyPDFDirectoryLoader("./us_census")

documents=loader.load()

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

final_documents=text_splitter.split_documents(documents)
final_documents[0]

Document(page_content='Health Insurance Coverage Status and Type \nby Geography: 2021 and 2022\nAmerican Community Survey Briefs\nACSBR-015Issued September 2023Douglas Conway and Breauna Branch\nINTRODUCTION\nDemographic shifts as well as economic and govern-\nment policy changes can affect people’s access to health coverage. For example, between 2021 and 2022, the labor market continued to improve, which may have affected private coverage in the United States \nduring that time.\n1 Public policy changes included \nthe renewal of the Public Health Emergency, which \nallowed Medicaid enrollees to remain covered under the Continuous Enrollment Provision.\n2 The American \nRescue Plan (ARP) enhanced Marketplace premium subsidies for those with incomes above 400 percent of the poverty level as well as for unemployed people.\n3', metadata={'source': 'us_census\\acsbr-015.pdf', 'page': 0})

In [3]:
len(final_documents)

316

In [7]:
## Embedding Using Huggingface
huggingface_embeddings=HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",      ##sentence-transformers/all-MiniLM-l6-v2
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True}

)

d:\Langchain\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\Langchain\.venv\Lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\TUFF\.cache\huggingface\hub\models--BAAI--bge-small-en-v1.5. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: 

In [8]:
import  numpy as np
print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)))
print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)).shape)

[-8.46568495e-02 -1.19099235e-02 -3.37892547e-02  2.94559300e-02
  5.19159660e-02  5.73839657e-02 -4.10017818e-02  2.74268147e-02
 -1.05128251e-01 -1.58055983e-02  7.94858858e-02  5.64318560e-02
 -1.31765436e-02 -3.41544151e-02  5.81603963e-03  4.72547971e-02
 -1.30746774e-02  3.12994118e-03 -3.44225690e-02  3.08406353e-02
 -4.09086123e-02  3.52737904e-02 -2.43761614e-02 -4.35831137e-02
  2.41503343e-02  1.31986756e-02 -4.84449510e-03  1.92347802e-02
 -5.43912724e-02 -1.42735004e-01  5.15527744e-03  2.93115862e-02
 -5.60810827e-02 -8.53535347e-03  3.14141214e-02  2.76736505e-02
 -2.06188280e-02  8.24231580e-02  4.15425487e-02  5.79655319e-02
 -3.71587314e-02  6.26160810e-03 -2.41390206e-02 -5.61793055e-03
 -2.51715314e-02  5.04968036e-03 -2.52801031e-02 -2.91945622e-03
 -8.24047904e-03 -5.69604486e-02  2.30822880e-02 -5.54219075e-03
  5.11555299e-02  6.09937869e-02  6.49766177e-02 -5.38514033e-02
  2.19109561e-02 -2.54194364e-02 -4.49223034e-02  4.22459170e-02
  4.75252122e-02  7.23253

In [9]:
## VectorStore Creation
vectorstore=FAISS.from_documents(final_documents[:120],huggingface_embeddings)

In [10]:
## Query using Similarity Search
query="WHAT IS HEALTH INSURANCE COVERAGE?"
relevant_docments=vectorstore.similarity_search(query)

print(relevant_docments[0].page_content)

2 U.S. Census Bureau
WHAT IS HEALTH INSURANCE COVERAGE?
This brief presents state-level estimates of health insurance coverage 
using data from the American Community Survey (ACS). The  
U.S. Census Bureau conducts the ACS throughout the year; the 
survey asks respondents to report their coverage at the time of 
interview. The resulting measure of health insurance coverage, 
therefore, reflects an annual average of current comprehensive 
health insurance coverage status.* This uninsured rate measures a 
different concept than the measure based on the Current Population 
Survey Annual Social and Economic Supplement (CPS ASEC). 
For reporting purposes, the ACS broadly classifies health insurance 
coverage as private insurance or public insurance. The ACS defines 
private health insurance as a plan provided through an employer 
or a union, coverage purchased directly by an individual from an 
insurance company or through an exchange (such as healthcare.


In [11]:
retriever=vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":3})
print(retriever)

tags=['FAISS', 'HuggingFaceBgeEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001BEBF766290> search_kwargs={'k': 3}


In [28]:
import os
os.environ['HUGGINGFACEHUB_API_TOKEN']="hf_STVjvhtHPIHNyClKcAnziTnypnHhPrza"

In [33]:
from langchain_community.llms import HuggingFaceEndpoint

In [38]:
hf = HuggingFaceEndpoint(
    repo_id="meta-llama/Meta-Llama-3-8B-Instruct",
    temperature=0.1,  # Pass temperature argument directly
    #max_length=500
)

query="What is the health insurance coverage?"
hf.invoke(query)

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\TUFF\.cache\huggingface\token
Login successful


' What is the health insurance coverage?\nHealth insurance coverage is a type of insurance that provides financial protection against medical expenses. It is a contract between an individual or a group and an insurance company, where the insurance company agrees to pay a portion of the medical expenses incurred by the policyholder in exchange for a premium payment.\nThere are several types of health insurance coverage, including:\n1. Individual health insurance: This type of coverage is purchased by an individual or a family and provides coverage for medical expenses.\n2. Group health insurance: This type of coverage is provided by an employer or a group and covers a group of individuals.\n3. Medicare: This is a government-funded health insurance program for individuals 65 years of age or older, or those with certain disabilities.\n4. Medicaid: This is a government-funded health insurance program for low-income individuals and families.\n5. Employer-sponsored health insurance: This typ

In [40]:
prompt_template="""
Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

{context}
Question:{question}

Helpful Answers:
 """

In [41]:
prompt=PromptTemplate(template=prompt_template,input_variables=["context","question"])

In [42]:
retrievalQA=RetrievalQA.from_chain_type(
    llm=hf,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt}
)

In [43]:
query="""DIFFERENCES IN THE
UNINSURED RATE BY STATE
IN 2022"""

In [44]:
# Call the QA chain with our query.
result = retrievalQA.invoke({"query": query})
print(result['result'])

 1. In 2022, uninsured rates at the time of interview ranged across states from a low of 2.4 percent in Massachusetts to a high of 16.6 percent in Texas, compared to the national rate of 8.0 percent.
  2. Ten of the 15 states with uninsured rates above the national average were not statistically different.
  3. Medicaid coverage accounted for a portion of that difference.
  4. Medicaid coverage was 22.7 percent in the group of states that expanded Medicaid eligibility and 18.0 percent in the group of non-expansion states.
  5. From 2021 to 2022, uninsured rates decreased across 27 states, while only Maine had an increase.
  6. The uninsured rate in Maine increased from 5.7 percent to 6.6 percent, although it remained below the national average.
  7. Maine’s uninsured rate was still below 8.0 percent.
  8. The private coverage rates were not statistically different in North Dakota and Utah.
  9. The percentage of uninsured people for the most populous metropolitan areas was not provided